# Model Interpretability: Eli5

**Paul's notes:** 

- Love the tables Eli5 gives us, but in order to view the model explanation table not as an HTML object but as a data frame, we can use: eli5.format_as_dataframe(explanation_object)
- Eli5's tables are NOT model agnostic (other methods we'll see are). For this reason, we should not compare the Eli5 explanation table of one model (say, a logit), with the explanationt able of another type of model (say, a neural net). 
- Takeaway is to use Eli5 if you have a whitebox model. If your model is NOT a whitebox model, you need to use Eli5's permutation importance, but then all you can see is magnitude of variable effects and not necessarily directionality. 

Note: This notebook uses features introduced in Python 3.6 and sklearn 0.20. 

First we'll need a few imports:

In [1]:
# Obviously
import pandas as pd

# Some sklearn tools for preprocessing. 
# ColumnTransformer was introduced in 0.20 so make sure you have this version
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

# In terms of metric we'll need balanced accuracy score and classification report
# More on that later...
from sklearn.metrics import balanced_accuracy_score, classification_report

# Our algorithms, from the easiest to the hardest to intepret.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [2]:
# this package was a huge pain for me to install
# after much trouble, I got it to work via the following link below:
# https://medium.com/tinghaochen/install-lightgbm-on-macos-high-sierra-1c640861e914
from lightgbm.sklearn import LGBMClassifier

/Users/pauljeffries/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


### The Dataset

The data is based on a dataset available [here](https://archive.ics.uci.edu/ml/datasets/bank+marketing). 

It consists of data from marketing campaigns of a Portuguese bank. We will build classifiers that given characteristics about an individual, and how they were approached by the marketing team, we can predict whether they will subscribe to a term deposit (column `y`). 

We will build multiple models with algorithms of different complexity. We'll keep the modelling part of the job simple though, so we can focus on interpreting our models and see what recommendation we can make to the domain experts (here the person in charge of the marketing campaign).

In [3]:
df = pd.read_csv("data/bank.csv")

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,no


In [4]:
df.y.value_counts()

no     36548
yes     4640
Name: y, dtype: int64

The dataset is imbalanced, we will need to keep that in mind when building our models...

In [5]:
# Get the feature matrix (X) and the target vector (y)

y = df["y"].map({"no": 0, "yes": 1})
X = df.drop("y", axis=1)

Let's look at the features in the X matrix and their types:

In [6]:
X.dtypes

age             int64
job            object
marital        object
education      object
default        object
housing        object
loan           object
contact        object
month          object
day_of_week    object
campaign        int64
pdays           int64
previous        int64
poutcome       object
dtype: object

1. `age` (numeric)
2. `job` : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. `marital` : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. `education` (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. `default`: has credit in default? (categorical: 'no','yes','unknown')
6. `housing`: has housing loan? (categorical: 'no','yes','unknown')
7. `loan`: has personal loan? (categorical: 'no','yes','unknown')
8. `contact`: contact communication type (categorical: 'cellular','telephone') 
9. `month`: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. `day_of_week`: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. `campaign`: number of contacts performed during this campaign and for this client (numeric, includes last contact)
12. `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
13. `previous`: number of contacts performed before this campaign and for this client (numeric)
14. `poutcome`: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

What features do you expect to be relevant to predict whether someone will subscribe to our plan?

To find those, we will train models and interpret them.

First, let's create a list of numerical and categorical features so we can easily select one or the other later:

In [7]:
# Some such as default would be binary features, but since
# they have a third class "unknown" we'll process them as non binary categorical
num_features = ["age", "campaign", "pdays", "previous"]

cat_features = ["job", "marital", "education","default", "housing", "loan",
                "contact", "month", "day_of_week", "poutcome"]

First, we'll define a new `ColumnTransformer` object that just keeps our numerical features as they are and apply one hot encoding on our categorical features. This transformer can be fitted on the training set and used to transform the test set in the same way:

In [8]:
preprocessor = ColumnTransformer([("numerical", "passthrough", num_features), 
                                  ("categorical", OneHotEncoder(sparse=False, handle_unknown="ignore"),
                                   cat_features)])

Let's split the data into training and test sets. Create `X_train`, `X_test`, `y_train` and `y_test` using `train_test_split` from sklearn:

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3, random_state=42)


Now we can create processed versions of X_train and X_test, ready to be used by our algorithms. To do so we will start by fitting the processor on X_train:

In [10]:
preprocessor.fit(X_train)

preprocessor.transform(X_train)

array([[ 50.,   2., 999., ...,   0.,   1.,   0.],
       [ 51.,   5., 999., ...,   0.,   1.,   0.],
       [ 46.,   2., 999., ...,   0.,   1.,   0.],
       ...,
       [ 35.,   3., 999., ...,   0.,   1.,   0.],
       [ 32.,   4., 999., ...,   0.,   1.,   0.],
       [ 31.,   3., 999., ...,   0.,   1.,   0.]])

When we call transform, the preprocessor returns a numpy array, which is great for Machine Learning algorithms to process ... but not so great for us humans to interpret. So we will wrap it up back to DataFrame with nice column names. To do so we will need to give nice names to the dummy feature generated by the one hot encoder. The code below extracts the list of categories, creates nice names for the dummy feature and create a new list `all_features` with good names for our columns:

In [11]:
# Get the list of categories generated by the process
ohe_categories = preprocessor.named_transformers_["categorical"].categories_

# Create nice names for our one hot encoded features
new_ohe_features = [f"{col}__{val}" for col, vals in zip(cat_features, ohe_categories) for val in vals]

# Create a new list with all names of features
all_features = num_features + new_ohe_features

Great, now we can create our preprocessed DataFrames with good column names:

In [12]:
X_train = pd.DataFrame(preprocessor.transform(X_train), columns=all_features)
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=all_features)

In [36]:
X_train.head()

,age,campaign,pdays,previous,job__admin.,job__blue-collar,job__entrepreneur,job__housemaid,job__management,job__retired,...,month__oct,month__sep,day_of_week__fri,day_of_week__mon,day_of_week__thu,day_of_week__tue,day_of_week__wed,poutcome__failure,poutcome__nonexistent,poutcome__success
0,50.0,2.0,999.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,51.0,5.0,999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,46.0,2.0,999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,46.0,1.0,999.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,25.0,5.0,999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Looks good!

Now we can define the 4 models we will interpret in this tutorial:

Let's start with a LogisticRegression:

In [37]:
# Logistic Regression
lr_model = LogisticRegression(class_weight="balanced", solver="liblinear", random_state=42)

Here we keep most parameters to default as we'll tune the model later. But defined `class_weight` to be balanced in order to take into account the imbalance in our dataset. 

Create `dt_model`, `rf_model` and `lgb_model` respectively using a Decistion Tree, a Random Forest and an LightGBM classifier, everytime make sure you set `class_weight` to `"balanced"`

In [38]:
# Decision Tree
dt_model = DecisionTreeClassifier(class_weight="balanced")

# Random Forest
rf_model = RandomForestClassifier(class_weight="balanced", n_estimators=150, n_jobs=-1)

# LightGBM -- Boosting model
lgb_model = LGBMClassifier(class_weight="balanced", n_jobs=-1)


## Using Eli5 to get importance of features globally and locally

### With Logistic Regression

First let's fine tune our logistic regression and evaluate its performance. Here we'll use ["balanced_accuracy"](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score) as metric to optimise so it takes into account the imbalance of classes.

In [39]:
gs = GridSearchCV(lr_model, {"C": [1., 1.3, 1.5]}, n_jobs=-1, cv=5, scoring="balanced_accuracy")
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [1.0, 1.3, 1.5]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='balanced_accuracy',
       verbose=0)

Let's see our best parameters and score

Check the best parameters and best score from the GridSearch object:

In [40]:
print(gs.best_params_)
print(gs.best_score_)


{'C': 1.0}
0.6970562866802703


Since `GridSearchCV` has a parameter `refit` that is `True` by default, it has re-fitted the model with best parameters on the whole training set. We can access it using `best_estimator_`

In [41]:
# LR model is our best trained model for LR
lr_model = gs.best_estimator_

Let's check that our model has properly been updated:

In [42]:
lr_model.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

Generate predictions on the test set; `y_pred`:

In [43]:
y_pred = lr_model.predict(X_test)


Compute balanced accuracy:

In [44]:
balanced_accuracy_score(y_test, y_pred)


0.7041273566363193

Check the classification report:

In [45]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.82      0.88     10965
           1       0.29      0.59      0.39      1392

   micro avg       0.79      0.79      0.79     12357
   macro avg       0.62      0.70      0.63     12357
weighted avg       0.87      0.79      0.82     12357



Great, so now we have a simple model that can predict if someone will subscribe to our plan. This isn't a great model in terms of accuracy, but it's simple enough that we can easily look inside the box and understand how it works.

For that we'll use `eli5` that will allow us to visualise the weights associated to each feature.

Import eli5 and use `show_weights` to visualise the weights of your model, don't forget to pass your column names as well so `eli5` can display them instead of the column number (check the documentation for `show_weights`)

In [46]:
import eli5

eli5.show_weights(lr_model, feature_names=all_features)


Weight?,Feature
+1.169,month__mar
+1.109,month__dec
+0.918,month__oct
+0.906,education__illiterate
+0.710,contact__cellular
+0.618,month__sep
+0.613,job__retired
+0.579,job__student
+0.558,default__no
+0.526,<BIAS>


That's easier to read. This table gives us the weight associated to each feature. The amplitude tells us how much of an impact a feature has on the predictions on average, the sign tells us in which direction. Here if the campaign is in March, it increases the probability of the prospect to subscribe to the plan significantly. Has the marketting team done something different in March? Or are prospects just more likely to subscribe in March? That's a question to ask to the marketting team, depending on the answer, this finding might or might not be useful.

We can also use `eli5` to explain a specific prediction, let's pick a row in the test data:

In [47]:
i = 4
X_test.iloc[[i]]

,age,campaign,pdays,previous,job__admin.,job__blue-collar,job__entrepreneur,job__housemaid,job__management,job__retired,...,month__oct,month__sep,day_of_week__fri,day_of_week__mon,day_of_week__thu,day_of_week__tue,day_of_week__wed,poutcome__failure,poutcome__nonexistent,poutcome__success
4,27.0,4.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [48]:
y_test.iloc[i]

1

Our prospect subsribed to the term deposit after the campaign! Let's see what our model would have predicted and how we could explain it to the domain expert.

In [49]:
eli5.show_prediction(lr_model, X_test.iloc[i],
                     feature_names=all_features, show_feature_values=True)

Here `eli5` does something really simple: knowing the weights associated to each feature and the actual value for all features for this specific observation, it can compute the contribution of each feature towards the prediction. 

For this prediction, it looks like the most important factor was that the prospect was contacted via phone (`contact__cellular`==1) and did not have a default (`default__no`==1). That's interesting information that you could share with the team member who was in touch with this prospect, to trigger a discussion on why those features were important.

### with a Decision Tree

`eli5` can also be used to intepret decision trees:

Use grid search to find the best parameters for your Decision Tree:

In [50]:
gs = GridSearchCV(dt_model, {"max_depth": [8, 10, 15], 
                             "min_samples_split": [.005, .01, .05]}, 
                  n_jobs=-1, cv=5, scoring="balanced_accuracy")

gs.fit(X_train, y_train)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [8, 10, 15], 'min_samples_split': [0.005, 0.01, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='balanced_accuracy', verbose=0)

Let's see our best parameters and score:

In [51]:
print(gs.best_params_)
print(gs.best_score_)


{'max_depth': 10, 'min_samples_split': 0.01}
0.6975556335157104


Store the best model `best_estimator_` into a new variable `dt_model` that we can try to interpret later:

In [52]:
dt_model = gs.best_estimator_


Check accuracy and classification report

In [53]:
y_pred = dt_model.predict(X_test)

print(balanced_accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.7103689049797947
              precision    recall  f1-score   support

           0       0.94      0.83      0.88     10965
           1       0.31      0.59      0.40      1392

   micro avg       0.80      0.80      0.80     12357
   macro avg       0.62      0.71      0.64     12357
weighted avg       0.87      0.80      0.83     12357



Unfortunately, for Decision Trees, `eli5` only gives the simple feature importance. It tells you how important different features are on average relative to each other but not how they impact the outcome.

Use `show_weights` on your dt_model:

In [54]:
eli5.show_weights(dt_model, feature_names=all_features)

Weight,Feature
0.3951,pdays
0.1622,contact__telephone
0.0916,age
0.0699,month__oct
0.0584,month__jun
0.0511,month__mar
0.0478,month__apr
0.0331,month__sep
0.0156,month__dec
0.0129,default__unknown


For this model, the most important feature seems to be `pdays` but we don't know if the more days the more likely it is that someone will subscribe, or the opposite. It can be useful to debug your model and know if it seems to pick up something that it shouldn't, but appart for that, it isn't too useful: you won't be able to properly explain what your model does to someone thanks to that.

## Limitations of the method...

So, `eli5`'s `show_weights` method is good, but for more complex models, such as trees the information provided starts to be less helpful. Since `show_weights` is accessing the internal weights of a model, it does not work with all algorithms, making it harder to compare different models you might have built.

`eli5` implements another technique called Permutation Importance that is model agnostic and works for any black box model. By shuffling at random the values of a feature, we can observe how that affects the predictions and quantify how important that feature is. If we repeat on all features, we can get the overall importance of each feature and compare them. Let's try to do that on our models.

### Using Permutation Importance for global and model agnostic feature importance

In [55]:
from eli5.sklearn import PermutationImportance

First will need to create a new instance of `PermutationImportance` that takes our trained model to be interpreted and the scoring method (`"balanced_accuracy"`) we which to use to quantify the importance of our features.

Create a PermutationImportance object to interpret your Decision Tree model, and call it `perm`

In [56]:
perm = PermutationImportance(dt_model, scoring="balanced_accuracy")


For the next step, we will need to `fit` our Permutation Importance. We could do so using the training set, but that would just tell us how important each feature was when trying to minimise the accuracy, not how your model will work on new data. 

To ensure your interpretation corresponds to how your model works when it is used on new data, you need to train the permutation importance on unseen data, using the test set for instance.

Call `fit` on your Permutation Importance object, then use `eli5`'s `show_weigths` as before, but passing the `perm` object instead of the model itself. This will plot your new feature importance:

In [57]:
perm.fit(X_test, y_test)
eli5.show_weights(perm, feature_names=all_features)


Weight,Feature
0.0483 ± 0.0062,contact__telephone
0.0374 ± 0.0055,month__jun
0.0340 ± 0.0020,pdays
0.0274 ± 0.0043,month__apr
0.0219 ± 0.0022,month__may
0.0207 ± 0.0067,age
0.0112 ± 0.0004,month__mar
0.0102 ± 0.0015,month__oct
0.0084 ± 0.0019,month__sep
0.0071 ± 0.0016,default__unknown


Again, here the feature importance is only given as an amplitude, we do not know in what direction it impacts the outcome. But the interpretation is quite interesting; when you will use your model on new data, to predict whether someone will subscribe or not to your plan, the most important thing it will need to get the prediction right is whether you contacted the person by telephone. You're not looking at what the model gave the most importance too whilst learning, but how it will give importance to features from now on based on what it has learnt. It also tells you that variables like whether the person got divorced will be pretty much irrelevant when your model is used.

Since Permutation Importance is model agnostic, we can use it on any other model that we have built, and compare the feature importance. Let's do that with a Random Forest and LightGBM model.

### with a Random Forest

Use grid search to find the best parameters for your `rf_model`

In [59]:
gs = GridSearchCV(rf_model, {"max_depth": [10, 15, 20],
                             "min_samples_split": [.005, .01, .05]}, 
                  n_jobs=-1, cv=5, scoring="balanced_accuracy")

gs.fit(X_train, y_train)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [10, 15, 20], 'min_samples_split': [0.005, 0.01, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='balanced_accuracy', verbose=0)

Check your best params and score:

In [60]:
print(gs.best_params_)
print(gs.best_score_)


{'max_depth': 15, 'min_samples_split': 0.01}
0.7134405012524075


Create `rf_model`, your model with best parameters:

In [61]:
rf_model = gs.best_estimator_


Generate predictions and check accuracy and classification report:

In [62]:
y_pred = rf_model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.725826395112977
              precision    recall  f1-score   support

           0       0.94      0.87      0.90     10965
           1       0.36      0.58      0.45      1392

   micro avg       0.84      0.84      0.84     12357
   macro avg       0.65      0.73      0.68     12357
weighted avg       0.88      0.84      0.85     12357



Build a new Permutation Importance object for your `rf_model` and fit it on the test data. What are the most important features here?

In [63]:
perm = PermutationImportance(rf_model, scoring="balanced_accuracy")
perm.fit(X_test, y_test)

eli5.show_weights(perm, feature_names=all_features)


Weight,Feature
0.0141 ± 0.0043,month__jun
0.0134 ± 0.0032,age
0.0112 ± 0.0038,month__apr
0.0108 ± 0.0040,contact__cellular
0.0095 ± 0.0014,month__oct
0.0087 ± 0.0012,month__mar
0.0074 ± 0.0042,contact__telephone
0.0070 ± 0.0024,month__aug
0.0066 ± 0.0013,month__nov
0.0064 ± 0.0030,poutcome__nonexistent


We find again that the feature that tells if prospects were contacted by telephone will have a big impact on whether our model will be right or wrong on new data. This seem to be particularly relevant, that's probably something we want to report to the marketing team.

### Let's train our LightGBM model as well

Now find the best parameters for your pipeline, and create `lgb_model` with the best parameters

In [64]:
gs = GridSearchCV(lgb_model, {"max_depth": [8, 10, 12],
                              "min_child_samples": [20, 40, 60],
                              "n_estimators": [25, 50, 75]},
                  n_jobs=-1, cv=5, scoring="balanced_accuracy")

gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)
lgb_model = gs.best_estimator_


{'max_depth': 10, 'min_child_samples': 40, 'n_estimators': 50}
0.7115638949798107


Generate predictions and check the accuracy and classification report:

In [65]:
y_pred = lgb_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.86      0.90     10965
           1       0.35      0.58      0.44      1392

   micro avg       0.83      0.83      0.83     12357
   macro avg       0.65      0.72      0.67     12357
weighted avg       0.88      0.83      0.85     12357



Build a new Permutation Importance object for your `lgb_model` and fit it on the test data. What are the most important features here?

In [66]:
perm = PermutationImportance(lgb_model, scoring="balanced_accuracy")
perm.fit(X_test, y_test)

eli5.show_weights(perm, feature_names=all_features)


Weight,Feature
0.0415 ± 0.0047,contact__cellular
0.0297 ± 0.0032,month__jun
0.0213 ± 0.0041,age
0.0155 ± 0.0023,pdays
0.0145 ± 0.0032,month__apr
0.0133 ± 0.0017,month__may
0.0121 ± 0.0022,month__oct
0.0081 ± 0.0029,month__mar
0.0077 ± 0.0038,month__aug
0.0058 ± 0.0020,month__sep


Quite similar features again, it looks like all our models, even the most complex ones seem to agree on what is important in the data.

Unfortunately, using Permutation Importance, we aren't able to tell much more than that. We cannot tell in what direction this will impact the outcome (which would be useful to make recommendations to the marketing team), nor can we explain a specific prediction... But at least this is model agnostic, so we can use this method for any model and compare the results as we did.

The next part of the tutorial is dedicated to LIME, a method that is also model agnostic and allows us to generate explainations of specific, local predictions.